## Phase 3 Anlaysis - January 2026
This notebook compares the performance of our trained DQN agent against the random baseline fro
m Phase 2.

In [ ]:
# Data manipulation
import pandas as pd

# Database connection
import psycopg2

# Visualization
import plotly.graph_objects as go

# Statistics

import plotly.io as pio
pio.renderers.default = 'notebook' 

print("✅ Imports loaded successfully!")

## Load Experiment Data

  We'll load episode metrics from two experiments:
  1. Random baseline (Phase 2)
  2. Trained DQN agent (Phase 3)


In [ ]:
# Database connection
conn = psycopg2.connect(
  host="localhost",
  database="mario_rl_db",
  user="mario_rl_user",
  password="Bingbongbing123!"
)

print("✅ Connected to database")

In [ ]:
# Find our experiments
query = """
SELECT experiment_id, experiment_name, algorithm, start_timestamp, total_episodes
FROM experiments
ORDER BY start_timestamp;
"""

experiments_df = pd.read_sql(query, conn)
print(experiments_df)

In [ ]:
# Load all episodes for both experiments
query_random = """
SELECT * FROM episodes 
WHERE experiment_id = 15
ORDER BY episode_number;
"""

query_dqn = """
SELECT * FROM episodes 
WHERE experiment_id = 14
ORDER BY episode_number;
"""

random_df = pd.read_sql(query_random, conn)
dqn_df = pd.read_sql(query_dqn, conn)

print(f"Random baseline: {len(random_df)} episodes")
print(f"DQN training: {len(dqn_df)} episodes")


## Summary Statistics

  Let's compare the two agents across key metrics:
  - Reward
  - Distance traveled
  - Score
  - Success rate (flag_get)


In [ ]:
# Calculate summary statistics for key metrics
metrics = ['reward', 'distance_traveled', 'score']

summary_data = []

for metric in metrics:
  summary_data.append({
      'Metric': metric,
      'Random Mean': random_df[metric].mean(),
      'Random Std': random_df[metric].std(),
      'DQN Mean': dqn_df[metric].mean(),
      'DQN Std': dqn_df[metric].std(),
      'Improvement': dqn_df[metric].mean() / random_df[metric].mean()
  })

summary_df = pd.DataFrame(summary_data)
print(summary_df.round(2))

In [ ]:
# Calculate success rates
random_success = (random_df['level_completed'].sum() / len(random_df)) * 100
dqn_success = (dqn_df['level_completed'].sum() / len(dqn_df)) * 100

print(f"\n{'='*50}")
print("SUCCESS RATE COMPARISON")
print(f"{'='*50}")
print(f"Random Agent:  {random_success:.1f}% ({random_df['level_completed'].sum()}/{len(random_df)} episodes)")
print(f"DQN Agent:     {dqn_success:.1f}% ({dqn_df['level_completed'].sum()}/{len(dqn_df)} episodes)")
print(f"{'='*50}")

## Visualizations

Visual comparison of agent performance.

In [ ]:
# Create comparison dataframe for plotting

fig = go.Figure()

fig.add_trace(go.Box(
  y=random_df['reward'],
  name='Random Agent',
  marker_color='coral',  # Changed to coral (reddish-orange)
  line=dict(width=2)
))

fig.add_trace(go.Box(
  y=dqn_df['reward'],
  name='DQN Agent',
  marker_color='mediumseagreen',  # Changed to medium sea green
  line=dict(width=2)
))

fig.update_layout(
  title='Reward Distribution: Random vs DQN',
  yaxis_title='Episode Reward',
  showlegend=True,
  height=500,
  plot_bgcolor='white',
  paper_bgcolor='white'
)

fig.show()

In [ ]:
# Learning curve with better colors
fig = go.Figure()

# Episode rewards - darker with more opacity
fig.add_trace(go.Scatter(
  x=dqn_df['episode_number'],
  y=dqn_df['reward'],
  mode='markers',
  name='Episode Reward',
  marker=dict(size=4, color='steelblue'),  # Changed to steelblue, bigger markers
  opacity=0.6  # Increased opacity
))

# Rolling average - bright contrasting color
window = 50
dqn_df['reward_ma'] = dqn_df['reward'].rolling(window=window, min_periods=1).mean()

fig.add_trace(go.Scatter(
  x=dqn_df['episode_number'],
  y=dqn_df['reward_ma'],
  mode='lines',
  name=f'{window}-Episode Moving Average',
  line=dict(color='green', width=4)  # Changed to green, thicker line
))

# Random baseline - bolder red
fig.add_hline(y=360.4, line_dash="dash", line_color="red",
            line_width=3,  # Thicker line
            annotation_text="Random Baseline (360.4)")

fig.update_layout(
  title='DQN Learning Curve: Reward Over Training',
  xaxis_title='Episode Number',
  yaxis_title='Reward',
  height=500,
  plot_bgcolor='white',  # White background
  paper_bgcolor='white'
)

fig.show()

